# <font color='DarkGreen'>Rafael Cavaletti Maia</font>
# <font color='DarkGreen'>Big Data Real-Time Analytics com Python e Spark - Data Science Academy</font>

## <font color='DarkGreen'>Projeto com Feedback 03 - Prevendo Nível de Satisfação dos Clientes do Santander</font>

www.datascienceacademy.com.br

## Parte 1 - Análise Exploratória e Limpeza dos Dados
## Parte 2 - Feature Engineering
## <font color='DarkBlue'>Parte 3 - Machine Learning</font>

## Versões utilizadas


In [1]:
# Versão da linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


## Pacotes utilizados

In [2]:
# Imports
import joblib
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Versão dos pacotes
#!pip install -q -U watermark
%reload_ext watermark
%watermark -a "Rafael Cavaletti Maia" --iversions

Author: Rafael Cavaletti Maia

matplotlib: 3.5.1
pandas    : 1.4.2
joblib    : 1.1.0
sklearn   : 1.1.3
numpy     : 1.21.5
seaborn   : 0.11.2



## Carregando os dados
Os dados carregados foram salvos no formato pickle após a engenharia de atributos realizado na etapa 2 do projeto.

In [4]:
# Carregando
X_treino = pd.read_pickle('pickle_files/X_over.pkl')
X_treino_log = pd.read_pickle('pickle_files/X_over_log.pkl')
X_teste = pd.read_pickle('pickle_files/X_teste.pkl')
X_teste_log = pd.read_pickle('pickle_files/X_teste_log.pkl')
y_treino = pd.read_pickle('pickle_files/y_over.pkl')
y_treino_log = pd.read_pickle('pickle_files/y_over_log.pkl')
y_teste = pd.read_pickle('pickle_files/y_teste.pkl')
y_teste_log = pd.read_pickle('pickle_files/y_teste_log.pkl')

## Criação dos modelos de Classificação
Iremos agora fazer a criação dos modelos de classificação do nosso problema.

###  Construção, Treinamento e Avaliação do Modelo 1 com Regressão Logística (Benchmark)
O modelo benchmark deve ser simples e de fácil explicação. Servirá como base para avaliar os outros modelos.

In [5]:
# Define lista de hiperparâmetros
tuned_params_v1 = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], 
                   'penalty': ['l1', 'l2']}

In [6]:
# Criaremos o modelo com GridSearch 
# Vários modelos serão criados com diferentes combinações de hiperparâmetros
modelo_v1 = GridSearchCV(LogisticRegression(), 
                         tuned_params_v1, 
                         scoring = 'roc_auc', 
                         n_jobs = -1)

modelo_v1_log = GridSearchCV(LogisticRegression(), 
                         tuned_params_v1, 
                         scoring = 'roc_auc', 
                         n_jobs = -1)

In [7]:
# Treinamento do modelo para os dados não transformados com log
modelo_v1.fit(X_treino, y_treino)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                               10000],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [8]:
# Treinamento do modelo para os dados transformados com log
modelo_v1_log.fit(X_treino_log, y_treino_log)

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,
                               10000],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [9]:
# Selecionando o melhor modelo para os dados não transformados com log
modelo_v1.best_estimator_

LogisticRegression(C=1)

In [10]:
# Selecionando o melhor modelo para os dados transformados com log
modelo_v1_log.best_estimator_

LogisticRegression(C=100)

In [11]:
# Previsões com os dados de teste
y_pred_v1 = modelo_v1.predict(X_teste)
y_pred_v1_log = modelo_v1_log.predict(X_teste_log)

In [12]:
# Obtemos as previsões no formato de probabilidade filtrando para a classe positiva
# Precisamos disso para calcula a Curva ROC
y_pred_proba_v1 = modelo_v1.predict_proba(X_teste)[:,1]
y_pred_proba_v1_log = modelo_v1_log.predict_proba(X_teste_log)[:,1]

In [13]:
# Confusion matrix para os resultados
print('Confusion matrix para valores não transformados:\n', confusion_matrix(y_teste, y_pred_v1))
print('')
print('Confusion matrix para valores transformados:\n', confusion_matrix(y_teste_log, y_pred_v1_log))


Confusion matrix para valores não transformados:
 [[14517  4997]
 [  233   591]]

Confusion matrix para valores transformados:
 [[14760  4754]
 [  227   597]]


In [14]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v1 = roc_auc_score(y_teste, y_pred_v1)
roc_auc_v1_log = roc_auc_score(y_teste_log, y_pred_v1_log)
print('Dados sem transformação:', roc_auc_v1)
print('')
print('Dados transformados:', roc_auc_v1_log)

Dados sem transformação: 0.730580223210421

Dados transformados: 0.7404472989767863


In [15]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v1, tpr_v1, thresholds = roc_curve(y_teste, y_pred_proba_v1)
fpr_v1_log, tpr_v1_log, thresholds = roc_curve(y_teste_log, y_pred_proba_v1_log)

# AUC em teste
auc_v1 = auc(fpr_v1, tpr_v1)
auc_v1_log = auc(fpr_v1_log, tpr_v1_log)
print('AUC em teste:')
print(50*'-')
print('Sem transformação: ', auc_v1)
print('Com transformação: ', auc_v1_log)

print('\n')

# Acurácia em teste
acuracia_v1 = accuracy_score(y_teste, y_pred_v1)
acuracia_v1_log = accuracy_score(y_teste_log, y_pred_v1_log)
print('Acurácia em teste:')
print(50*'-')
print('Sem transformação',acuracia_v1)
print('Com transformação',acuracia_v1_log)

AUC em teste:
--------------------------------------------------
Sem transformação:  0.7859022486718522
Com transformação:  0.7901196589254814


Acurácia em teste:
--------------------------------------------------
Sem transformação 0.7428459042187039
Com transformação 0.7550889959681385


### Feature Importance

In [16]:
# Construindo o modelo novamente com os melhores hiperparâmetros
# Isso é necessário pois a versão final não deve ter o GridSearchCV
modelo_v1 = LogisticRegression(C = 1000)
modelo_v1_log = LogisticRegression(C = 10000)

modelo_v1.fit(X_treino, y_treino)
modelo_v1_log.fit(X_treino_log, y_treino_log)

LogisticRegression(C=10000)

In [17]:
# Obtemos os coeficientes pelo maior valor usando np.argsort
indices = np.argsort(-abs(modelo_v1.coef_[0,:]))
indices_log = np.argsort(-abs(modelo_v1_log.coef_[0,:]))

print("Variáveis mais importantes para o resultado do modelo_v1:")
print(70*'-')
for feature in X_treino.columns[indices]:
    print(feature)
    
print('\n')


print("Variáveis mais importantes para o resultado do modelo_v1_log:")
print(70*'-')
for feature_log in X_treino_log.columns[indices_log]:
    print(feature_log)

Variáveis mais importantes para o resultado do modelo_v1:
----------------------------------------------------------------------
var15
ind_var30
num_var30_0
ind_var12_0
ind_var13_0
saldo_medio_var5_ult3
var38
ind_var43_emit_ult1
ind_var13_largo_0
ind_var43_recib_ult1
num_med_var22_ult3
imp_op_var41_efect_ult1
saldo_medio_var12_hace3
saldo_var30
ind_var39_0
saldo_medio_var5_hace3
imp_op_var39_comer_ult3
ind_var8_0
num_var22_ult1
ind_var14_0
num_var43_recib_ult1
var36
imp_trans_var37_ult1
num_aport_var13_hace3
saldo_medio_var13_corto_hace3


Variáveis mais importantes para o resultado do modelo_v1_log:
----------------------------------------------------------------------
var15
ind_var13_0
ind_var12_0
num_aport_var13_hace3
saldo_medio_var13_corto_hace3
saldo_medio_var5_ult3
ind_var30
num_var30_0
var38
ind_var43_emit_ult1
imp_trans_var37_ult1
num_med_var22_ult3
ind_var43_recib_ult1
saldo_medio_var5_hace3
imp_op_var41_efect_ult1
imp_op_var39_comer_ult3
var36
ind_var39_0
ind_var8_0
saldo_va

In [18]:
# Salva os modelos em disco
with open('modelos/modelo_v1.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v1, 'modelos/modelo_v1.pkl') 

with open('modelos/modelo_v1_log.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v1_log, 'modelos/modelo_v1_log.pkl') 

In [19]:
# Cria um dataframe para receber as métricas de cada modelo
df_modelos = pd.DataFrame()

In [20]:
# Dicionário com as métricas do modelo_v1
dict_modelo_v1 = {'Nome': 'modelo_v1', 
                  'Algoritmo': 'Regressão Logística', 
                  'ROC_AUC Score': roc_auc_v1,
                  'AUC Score': auc_v1,
                  'Acurácia': acuracia_v1}

# Dicionário com as métricas do modelo_v1_log
dict_modelo_v1_log = {'Nome': 'modelo_v1_log', 
                  'Algoritmo': 'Regressão Logística', 
                  'ROC_AUC Score': roc_auc_v1_log,
                  'AUC Score': auc_v1_log,
                  'Acurácia': acuracia_v1_log}

In [21]:
# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v1, ignore_index = True)
df_modelos = df_modelos.append(dict_modelo_v1_log, ignore_index = True)

# Visualiza
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,0.730580,0.785902,0.742846
1,modelo_v1_log,Regressão Logística,0.740447,0.790120,0.755089


### Construção, Treinamento e Avaliação do Modelo 2 com Random Forest

In [22]:
# Grid de hiperparâmetros
tuned_params_v2 = {'n_estimators': [100, 200, 300, 400, 500], 
                   'min_samples_split': [2, 5, 10], 
                   'min_samples_leaf': [1, 2, 4]}

tuned_params_v2_log = {'n_estimators': [100, 200, 300, 400, 500], 
                   'min_samples_split': [2, 5, 10], 
                   'min_samples_leaf': [1, 2, 4]}

In [23]:
# Cria o modelo com RandomizedSearchCV para buscar a melhor combinação de hiperparâmetros
modelo_v2 = RandomizedSearchCV(RandomForestClassifier(), 
                               tuned_params_v2, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs  = -1)

modelo_v2_log = RandomizedSearchCV(RandomForestClassifier(), 
                               tuned_params_v2, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs  = -1)

In [24]:
# Treina o modelo
modelo_v2.fit(X_treino, y_treino)
modelo_v2_log.fit(X_treino_log, y_treino_log)

RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=15, n_jobs=-1,
                   param_distributions={'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='roc_auc')

In [25]:
# Extrai o melhor modelo
print('Estimator modelo sem transformação:', modelo_v2.best_estimator_)
print('Estimator modelo com transformação:', modelo_v2_log.best_estimator_)

Estimator modelo sem transformação: RandomForestClassifier(min_samples_leaf=2, n_estimators=500)
Estimator modelo com transformação: RandomForestClassifier(min_samples_leaf=2, n_estimators=200)


In [26]:
# Previsões em teste
y_pred_v2 = modelo_v2.predict(X_teste)
y_pred_v2_log = modelo_v2_log.predict(X_teste_log)

In [27]:
# Obtém as previsões para a classe positiva
y_pred_proba_v2 = modelo_v2.predict_proba(X_teste)[:,1]
y_pred_proba_v2_log = modelo_v2_log.predict_proba(X_teste)[:,1]

In [28]:
# Confusion matrix para os resultados
print('Confusion matrix para valores não transformados - Random Forest:\n', confusion_matrix(y_teste, y_pred_v2))
print(80*'-')
print('Confusion matrix para valores transformados - Random Forest:\n', confusion_matrix(y_teste_log, y_pred_v2_log))

Confusion matrix para valores não transformados - Random Forest:
 [[17346  2168]
 [  413   411]]
--------------------------------------------------------------------------------
Confusion matrix para valores transformados - Random Forest:
 [[17385  2129]
 [  428   396]]


In [29]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v2 = roc_auc_score(y_teste, y_pred_v2)
roc_auc_v2_log = roc_auc_score(y_teste_log, y_pred_v2_log)
print('Dados sem transformação:', roc_auc_v2)
print('')
print('Dados transformados:', roc_auc_v2_log)

Dados sem transformação: 0.6938433422456967

Dados transformados: 0.6857406830644864


In [30]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v2, tpr_v2, thresholds = roc_curve(y_teste, y_pred_proba_v2)
fpr_v2_log, tpr_v2_log, thresholds = roc_curve(y_teste_log, y_pred_proba_v2_log)

# AUC em teste
auc_v2 = auc(fpr_v2, tpr_v2)
auc_v2_log = auc(fpr_v2_log, tpr_v2_log)
print('AUC em teste:')
print(50*'-')
print('Sem transformação: ', auc_v2)
print('Com transformação: ', auc_v2_log)

print('\n')

# Acurácia em teste
acuracia_v2 = accuracy_score(y_teste, y_pred_v2)
acuracia_v2_log = accuracy_score(y_teste_log, y_pred_v2_log)
print('Acurácia em teste:')
print(50*'-')
print('Sem transformação',acuracia_v2)
print('Com transformação',acuracia_v2_log)

AUC em teste:
--------------------------------------------------
Sem transformação:  0.8058506476803808
Com transformação:  0.8072561608742939


Acurácia em teste:
--------------------------------------------------
Sem transformação 0.8730946995771462
Com transformação 0.8742747566132363


### Feature Importance

In [31]:
# Recria o modelo com os melhores hiperparâmetros
modelo_v2 = RandomForestClassifier(n_estimators = 500, min_samples_leaf = 2)
modelo_v2.fit(X_treino, y_treino)

modelo_v2_log = RandomForestClassifier(n_estimators = 500, min_samples_split=5, min_samples_leaf = 2)
modelo_v2_log.fit(X_treino_log, y_treino_log)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=5,
                       n_estimators=500)

In [32]:
# Variáveis mais relevantes
indices = np.argsort(-modelo_v2.feature_importances_)
indices_log = np.argsort(-modelo_v2_log.feature_importances_)

print("Variáveis mais importantes para o resultado do modelo_v2:")
print(80*'-')
for feature in X_treino.columns[indices]:
    print(feature)

print('\n')

print("Variáveis mais importantes para o resultado do modelo_v2:")
print(80*'-')
for feature_log in X_treino_log.columns[indices_log]:
    print(feature_log)

Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------------------------------------
var15
var38
saldo_medio_var5_ult3
saldo_var30
saldo_medio_var5_hace3
ind_var30
var36
num_med_var22_ult3
num_var22_ult1
imp_op_var39_comer_ult3
imp_op_var41_efect_ult1
imp_trans_var37_ult1
num_var30_0
ind_var39_0
ind_var43_emit_ult1
num_var43_recib_ult1
ind_var43_recib_ult1
ind_var12_0
ind_var8_0
ind_var13_0
saldo_medio_var12_hace3
ind_var14_0
num_aport_var13_hace3
saldo_medio_var13_corto_hace3
ind_var13_largo_0


Variáveis mais importantes para o resultado do modelo_v2:
--------------------------------------------------------------------------------
var15
var38
saldo_var30
saldo_medio_var5_ult3
saldo_medio_var5_hace3
ind_var30
var36
num_med_var22_ult3
num_var22_ult1
imp_op_var39_comer_ult3
imp_op_var41_efect_ult1
num_var30_0
ind_var39_0
imp_trans_var37_ult1
num_var43_recib_ult1
ind_var43_emit_ult1
ind_var12_0
ind_var43_recib_ult1
ind_var13_0
ind_var

In [33]:
# Salva os modelos em disco
with open('modelos/modelo_v2.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v2, 'modelos/modelo_v2.pkl') 

with open('modelos/modelo_v2_log.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v2_log, 'modelos/modelo_v2_log.pkl') 

In [34]:
# Dicionário com as métricas do modelo_v2
dict_modelo_v2 = {'Nome': 'modelo_v2', 
                  'Algoritmo': 'Random Forest', 
                  'ROC_AUC Score': roc_auc_v2,
                  'AUC Score': auc_v2,
                  'Acurácia': acuracia_v2}

dict_modelo_v2_log = {'Nome': 'modelo_v2_log', 
                  'Algoritmo': 'Random Forest', 
                  'ROC_AUC Score': roc_auc_v2_log,
                  'AUC Score': auc_v2_log,
                  'Acurácia': acuracia_v2_log}

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v2, ignore_index = True)
df_modelos = df_modelos.append(dict_modelo_v2_log, ignore_index = True)

# Visualiza o dataframe
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,0.730580,0.785902,0.742846
1,modelo_v1_log,Regressão Logística,0.740447,0.790120,0.755089
2,modelo_v2,Random Forest,0.693843,0.805851,0.873095
3,modelo_v2_log,Random Forest,0.685741,0.807256,0.874275


### Construção, Treinamento e Avaliação do Modelo 3 com KNN

In [35]:
# Lista de possíveis valores de K
vizinhos = list(range(1, 20, 2))

# Lista para os scores
cv_scores = []
cv_scores_log = []

In [36]:
# Validação cruzada para determinar o melhor valor de k no modelo com os dados não transformados
for k in vizinhos:
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, X_treino, y_treino, cv = 5, scoring = 'accuracy')
    cv_scores.append(scores.mean())

In [37]:
# Ajustando o erro de classificação
erro = [1 - x for x in cv_scores]

In [38]:
# Determinando o melhor valor de k (com menor erro)
optimal_k = vizinhos[erro.index(min(erro))]
print('O valor ideal de k para o modelo sem transformar os dados é %d' % optimal_k)

O valor ideal de k para o modelo sem transformar os dados é 3


In [39]:
# Validação cruzada para determinar o melhor valor de k no modelo com os dados transformados
for k_log in vizinhos:
    knn_log = KNeighborsClassifier(n_neighbors = k_log)
    scores_log = cross_val_score(knn_log, X_treino_log, y_treino_log, cv = 5, scoring = 'accuracy')
    cv_scores_log.append(scores_log.mean())

In [40]:
# Ajustando o erro de classificação
erro_log = [1 - x for x in cv_scores_log]

In [41]:
# Determinando o melhor valor de k (com menor erro)
optimal_k_log = vizinhos[erro.index(min(erro))]
print('O valor ideal de k para o modelo sem transformar os dados é %d' % optimal_k_log)

O valor ideal de k para o modelo sem transformar os dados é 3


In [42]:
# Criamos o modelo versão 3
modelo_v3 = KNeighborsClassifier(n_neighbors = optimal_k)
modelo_v3_log = KNeighborsClassifier(n_neighbors = optimal_k_log)

In [43]:
# Treinamento
modelo_v3.fit(X_treino, y_treino)
modelo_v3_log.fit(X_treino_log, y_treino_log)

KNeighborsClassifier(n_neighbors=3)

In [44]:
# Previsões
y_pred_v3 = modelo_v3.predict(X_teste)
y_pred_v3_log = modelo_v3_log.predict(X_teste_log)

In [45]:
# Confusion matrix para os resultados
print('Confusion matrix para valores não transformados - Random Forest:\n', confusion_matrix(y_teste, y_pred_v3))
print(80*'-')
print('Confusion matrix para valores transformados - Random Forest:\n', confusion_matrix(y_teste_log, y_pred_v3_log))

Confusion matrix para valores não transformados - Random Forest:
 [[16365  3149]
 [  487   337]]
--------------------------------------------------------------------------------
Confusion matrix para valores transformados - Random Forest:
 [[16687  2827]
 [  498   326]]


In [46]:
# Obtém as previsões para a classe positiva
y_pred_proba_v3 = modelo_v3.predict_proba(X_teste)[:,1]
y_pred_proba_v3_log = modelo_v3_log.predict_proba(X_teste)[:,1]

In [47]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v3 = roc_auc_score(y_teste, y_pred_v3)
roc_auc_v3_log = roc_auc_score(y_teste_log, y_pred_v3_log)
print('Dados sem transformação:', roc_auc_v3)
print('')
print('Dados transformados:', roc_auc_v3_log)

Dados sem transformação: 0.6238046296858317

Dados transformados: 0.6253803592342465


In [48]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v3, tpr_v3, thresholds = roc_curve(y_teste, y_pred_proba_v3)
fpr_v3_log, tpr_v3_log, thresholds = roc_curve(y_teste_log, y_pred_proba_v3_log)

# AUC em teste
auc_v3 = auc(fpr_v3, tpr_v3)
auc_v3_log = auc(fpr_v3_log, tpr_v3_log)
print('AUC em teste:')
print(50*'-')
print('Sem transformação: ', auc_v3)
print('Com transformação: ', auc_v3_log)

print('\n')

# Acurácia em teste
acuracia_v3 = accuracy_score(y_teste, y_pred_v3)
acuracia_v3_log = accuracy_score(y_teste_log, y_pred_v3_log)
print('Acurácia em teste:')
print(50*'-')
print('Sem transformação',acuracia_v3)
print('Com transformação',acuracia_v3_log)

AUC em teste:
--------------------------------------------------
Sem transformação:  0.6618785206239781
Com transformação:  0.6904445501412478


Acurácia em teste:
--------------------------------------------------
Sem transformação 0.8212213590323533
Com transformação 0.8365129314583538


In [49]:
# Salva os modelos em disco
with open('modelos/modelo_v3.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v3, 'modelos/modelo_v3.pkl') 

with open('modelos/modelo_v3_log.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v3_log, 'modelos/modelo_v3_log.pkl')

In [50]:
# Dicionário com as métricas do modelo_v3
dict_modelo_v3 = {'Nome': 'modelo_v3', 
                  'Algoritmo': 'KNN', 
                  'ROC_AUC Score': roc_auc_v3,
                  'AUC Score': auc_v3,
                  'Acurácia': acuracia_v3}

dict_modelo_v3_log = {'Nome': 'modelo_v3_log', 
                      'Algoritmo': 'KNN', 
                      'ROC_AUC Score': roc_auc_v3_log,
                      'AUC Score': auc_v3_log,
                      'Acurácia': acuracia_v3_log}

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v3, ignore_index = True)
df_modelos = df_modelos.append(dict_modelo_v3_log, ignore_index = True)

# Visualiza o dataframe
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,0.730580,0.785902,0.742846
1,modelo_v1_log,Regressão Logística,0.740447,0.790120,0.755089
2,modelo_v2,Random Forest,0.693843,0.805851,0.873095
3,modelo_v2_log,Random Forest,0.685741,0.807256,0.874275
4,modelo_v3,KNN,0.623805,0.661879,0.821221
5,modelo_v3_log,KNN,0.625380,0.690445,0.836513


### Construção, Treinamento e Avaliação do Modelo 4 com Decision Tree

In [51]:
# Hiperparâmetros
tuned_params_v4 = {'min_samples_split': [2, 3, 4, 5, 7], 
                   'min_samples_leaf': [1, 2, 3, 4, 6], 
                   'max_depth': [2, 3, 4, 5, 6, 7]}

tuned_params_v4_log = {'min_samples_split': [2, 3, 4, 5, 7], 
                       'min_samples_leaf': [1, 2, 3, 4, 6], 
                       'max_depth': [2, 3, 4, 5, 6, 7]}

In [52]:
# Cria o modelo com RandomizedSearchCV
modelo_v4 = RandomizedSearchCV(DecisionTreeClassifier(), 
                               tuned_params_v4, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs = -1)

modelo_v4_log = RandomizedSearchCV(DecisionTreeClassifier(), 
                               tuned_params_v4_log, 
                               n_iter = 15, 
                               scoring = 'roc_auc', 
                               n_jobs = -1)

In [53]:
# Treinamento
modelo_v4.fit(X_treino, y_treino)
modelo_v4_log.fit(X_treino_log, y_treino_log)

RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=15, n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 4, 5, 6, 7],
                                        'min_samples_leaf': [1, 2, 3, 4, 6],
                                        'min_samples_split': [2, 3, 4, 5, 7]},
                   scoring='roc_auc')

In [54]:
# Extrai o melhor modelo
print('Estimator modelo sem transformação:', modelo_v4.best_estimator_)
print('Estimator modelo com transformação:', modelo_v4_log.best_estimator_)

Estimator modelo sem transformação: DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, min_samples_split=5)
Estimator modelo com transformação: DecisionTreeClassifier(max_depth=7, min_samples_leaf=6, min_samples_split=7)


In [55]:
# Previsões de classe
y_pred_v4 = modelo_v4.predict(X_teste)
y_pred_v4_log = modelo_v4_log.predict(X_teste_log)

In [56]:
# Previsões de probabilidade
y_pred_proba_v4 = modelo_v4.predict_proba(X_teste)[:,1]
y_pred_proba_v4_log = modelo_v4_log.predict_proba(X_teste_log)[:,1]

In [57]:
# Confusion matrix para os resultados
print('Confusion matrix para valores não transformados - Decision Tree:\n', confusion_matrix(y_teste, y_pred_v4))
print(80*'-')
print('Confusion matrix para valores transformados - Decision Tree:\n', confusion_matrix(y_teste_log, y_pred_v4_log))

Confusion matrix para valores não transformados - Decision Tree:
 [[15314  4200]
 [  264   560]]
--------------------------------------------------------------------------------
Confusion matrix para valores transformados - Decision Tree:
 [[15110  4404]
 [  243   581]]


In [58]:
# Calcula a métrica global AUC (Area Under The Curve) com dados reais e previsões em teste
roc_auc_v4 = roc_auc_score(y_teste, y_pred_v4)
roc_auc_v4_log = roc_auc_score(y_teste_log, y_pred_v4_log)
print('Dados sem transformação:', roc_auc_v4)
print('')
print('Dados transformados:', roc_auc_v4_log)

Dados sem transformação: 0.7321907796344372

Dados transformados: 0.7397064815800654


In [59]:
# Calcula a curva ROC com dados e previsões em teste
fpr_v4, tpr_v4, thresholds = roc_curve(y_teste, y_pred_proba_v4)
fpr_v4_log, tpr_v4_log, thresholds = roc_curve(y_teste_log, y_pred_proba_v4_log)

# AUC em teste
auc_v4 = auc(fpr_v4, tpr_v4)
auc_v4_log = auc(fpr_v4_log, tpr_v4_log)
print('AUC em teste:')
print(50*'-')
print('Sem transformação: ', auc_v4)
print('Com transformação: ', auc_v4_log)

print('\n')

# Acurácia em teste
acuracia_v4 = accuracy_score(y_teste, y_pred_v4)
acuracia_v4_log = accuracy_score(y_teste_log, y_pred_v4_log)
print('Acurácia em teste:')
print(50*'-')
print('Sem transformação',acuracia_v4)
print('Com transformação',acuracia_v4_log)

AUC em teste:
--------------------------------------------------
Sem transformação:  0.8054341555626976
Com transformação:  0.8006281089205558


Acurácia em teste:
--------------------------------------------------
Sem transformação 0.7805093912872455
Com transformação 0.7715114563870588


### Feature Importance

In [60]:
# Recriando o modelo
modelo_v4 = DecisionTreeClassifier(min_samples_split = 3, min_samples_leaf = 3, max_depth = 6)
modelo_v4.fit(X_treino, y_treino)

modelo_v4_log = DecisionTreeClassifier(min_samples_split = 4, min_samples_leaf = 4, max_depth = 7)
modelo_v4_log.fit(X_treino_log, y_treino_log)

DecisionTreeClassifier(max_depth=7, min_samples_leaf=4, min_samples_split=4)

In [61]:
# Variáveis mais relevantes
indices = np.argsort(-modelo_v4.feature_importances_)
indices_log = np.argsort(-modelo_v4_log.feature_importances_)

print("Variáveis mais importantes para o resultado do modelo_v4:")
print(80*'-')
for feature in X_treino.columns[indices]:
    print(feature)

print('\n')

print("Variáveis mais importantes para o resultado do modelo_v4:")
print(80*'-')
for feature_log in X_treino_log.columns[indices_log]:
    print(feature_log)

Variáveis mais importantes para o resultado do modelo_v4:
--------------------------------------------------------------------------------
var15
saldo_var30
ind_var30
num_var22_ult1
var38
num_med_var22_ult3
ind_var43_emit_ult1
var36
num_var30_0
saldo_medio_var5_ult3
saldo_medio_var5_hace3
ind_var12_0
ind_var43_recib_ult1
imp_op_var41_efect_ult1
ind_var39_0
num_aport_var13_hace3
imp_trans_var37_ult1
ind_var14_0
imp_op_var39_comer_ult3
saldo_medio_var12_hace3
saldo_medio_var13_corto_hace3
ind_var13_largo_0
ind_var13_0
ind_var8_0
num_var43_recib_ult1


Variáveis mais importantes para o resultado do modelo_v4:
--------------------------------------------------------------------------------
var15
saldo_var30
var38
ind_var30
num_var30_0
ind_var43_emit_ult1
saldo_medio_var5_hace3
num_med_var22_ult3
num_var22_ult1
var36
saldo_medio_var5_ult3
imp_op_var39_comer_ult3
num_var43_recib_ult1
imp_op_var41_efect_ult1
ind_var39_0
ind_var13_0
ind_var12_0
ind_var8_0
imp_trans_var37_ult1
saldo_medio_var12

In [62]:
# Salva os modelos em disco
with open('modelos/modelo_v4.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v4, 'modelos/modelo_v4.pkl') 

with open('modelos/modelo_v4_log.pkl', 'wb') as pickle_file:
      joblib.dump(modelo_v4_log, 'modelos/modelo_v4_log.pkl')

In [68]:
# Dicionário com as métricas do modelo_v4
dict_modelo_v4 = {'Nome': 'modelo_v4', 
                  'Algoritmo': 'Decision Tree', 
                  'ROC_AUC Score': roc_auc_v4,
                  'AUC Score': auc_v4,
                  'Acurácia': acuracia_v4}

dict_modelo_v4_log = {'Nome': 'modelo_v4_log', 
                      'Algoritmo': 'Decision Tree', 
                      'ROC_AUC Score': roc_auc_v4_log,
                      'AUC Score': auc_v4_log,
                      'Acurácia': acuracia_v4_log}

# Adiciona o dict ao dataframe
df_modelos = df_modelos.append(dict_modelo_v4, ignore_index = True)
df_modelos = df_modelos.append(dict_modelo_v4_log, ignore_index = True)

# Visualiza o dataframe
display(df_modelos)

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
0,modelo_v1,Regressão Logística,0.730580,0.785902,0.742846
1,modelo_v1_log,Regressão Logística,0.740447,0.790120,0.755089
2,modelo_v2,Random Forest,0.693843,0.805851,0.873095
3,modelo_v2_log,Random Forest,0.685741,0.807256,0.874275
4,modelo_v3,KNN,0.623805,0.661879,0.821221
5,modelo_v3_log,KNN,0.625380,0.690445,0.836513
6,modelo_v4,Decision Tree,0.732191,0.805434,0.780509
7,modelo_v4_log,Decision Tree,0.739706,0.800628,0.771511


### Selecionando o melhor modelo

Para a seleção do melhor modelo usaremos a métrica AUC Score, pois foram utilizados diferentes modelos e queremos escolher o melhor dentre eles.

In [69]:
# Selecionando o melhor modelo
df_melhor_modelo = df_modelos[df_modelos['AUC Score'] == df_modelos['AUC Score'].max()]
df_melhor_modelo

,Nome,Algoritmo,ROC_AUC Score,AUC Score,Acurácia
3,modelo_v2_log,Random Forest,0.685741,0.807256,0.874275


## Conclusão
Para os modelos selecionados, aquele cujo algoritmo **Random Forest** aplicado ao dataset com os dados cuja transformação logaritmica foi aplicada apresentou maior valor de AUC Score, sendo assim, foi o que obteve a melhor performance. 
Outros algoritmos poderiam ter sido avaliados, como o **SVM** e o **XGBoost**(que apresentou melhor performance de acordo com o Kaggle). 
Foram realizadas tentativas de aplicar o SVM, mas o programa ficou uma tarde inteira rodando e ainda não tinha completado o fit dos dados (provavelmente pela alta dimensionalidade). Isso sugere que deve ser feito um trabalho mais meticuloso na etapa de Feature Engineering e pré processamento dos dados. Esse projeto será revisitado futuramente com maior bagagem de conhecimento para fazer um melhor pré processamento na busca de encontrar um novo melhor modelo que se encaixe nos dados selecionados.